In [208]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
# PyTorch related imports would go here
import numpy as np

In [209]:
from sklearn.feature_extraction.text import TfidfVectorizer
from us import states
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import re

In [210]:
state_names = [state.name.lower() for state in states.STATES]
state_abbrs = [state.abbr for state in states.STATES]
city_df = pd.read_csv('PM25-Speciated/worldcities.csv')
city_names = city_df['city_ascii'].str.lower().tolist()  # First column
country_names = city_df['country'].str.lower().tolist()  # First column

In [211]:
stop_words = TfidfVectorizer(stop_words='english').get_stop_words()
stop_words = list(list(stop_words)+ city_names + country_names + state_names + state_abbrs)

# Original Data

In [212]:
DATASET_NAME = "SpeciateV5.3_PM_AE6.csv"

df = pd.read_csv('PM25-Speciated/specviate_v5_3datasets/'+DATASET_NAME, encoding='ISO-8859-1')

In [213]:
df

,Unnamed: 0,PROFILE_CODE,WEIGHT_PERCENT,UNCERTAINTY_PERCENT,SPECIES_NAME,SYMBOL,PROFILE_NAME,PROFILE_TYPE,TOTAL,REGION,CATEGORY_LEVEL_1_Generation_Mechanism,CATEGORY_LEVEL_2_Sector_Equipment,CATEGORY_LEVEL_3_.Fuel_Product
0,74033,3766AE6,0.001500,0.001200,Tin,Sn,Forest Fire - AE6,PM-AE6,100.0,Northwestern Colorado,Combustion,Biomass Burning; Wildfire,Pinyon-juniper
1,74034,3766AE6,0.079200,0.021300,Sulfur,S,Forest Fire - AE6,PM-AE6,100.0,Northwestern Colorado,Combustion,Biomass Burning; Wildfire,Pinyon-juniper
2,74035,3766AE6,0.133200,0.062900,Sulfate,SO4=,Forest Fire - AE6,PM-AE6,100.0,Northwestern Colorado,Combustion,Biomass Burning; Wildfire,Pinyon-juniper
3,74038,3766AE6,0.000100,0.000200,Vanadium,V,Forest Fire - AE6,PM-AE6,100.0,Northwestern Colorado,Combustion,Biomass Burning; Wildfire,Pinyon-juniper
4,74039,3766AE6,0.120300,0.095900,Potassium ion,K+,Forest Fire - AE6,PM-AE6,100.0,Northwestern Colorado,Combustion,Biomass Burning; Wildfire,Pinyon-juniper
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4131,347743,SUGP02,3.361345,0.504202,Sulfate,SO4=,Sugar Cane Pre-Harvest Burning Mexico,PM-AE6,100.0,"Mexico - Chiapas, Jalisco, Veracruz, and Morelos",Combustion,Biomass Burning; Agriculture,Pre-harvest Sugar Cane
4132,347745,SUGP02,0.168067,0.084034,Magnesium,Mg,Sugar Cane Pre-Harvest Burning Mexico,PM-AE6,100.0,"Mexico - Chiapas, Jalisco, Veracruz, and Morelos",Combustion,Biomass Burning; Agriculture,Pre-harvest Sugar Cane
4133,347747,SUGP02,0.168067,0.000000,Nitrate,NO3-,Sugar Cane Pre-Harvest Burning Mexico,PM-AE6,100.0,"Mexico - Chiapas, Jalisco, Veracruz, and Morelos",Combustion,Biomass Burning; Agriculture,Pre-harvest Sugar Cane
4134,347750,SUGP02,2.184874,0.420168,Ammonium,NH4+,Sugar Cane Pre-Harvest Burning Mexico,PM-AE6,100.0,"Mexico - Chiapas, Jalisco, Veracruz, and Morelos",Combustion,Biomass Burning; Agriculture,Pre-harvest Sugar Cane


In [214]:
df = df.pivot_table(index=['PROFILE_CODE', 'PROFILE_NAME'], columns='SPECIES_NAME', values='WEIGHT_PERCENT', aggfunc='first').reset_index()
# Fill NaN values with 0 (or any other value you prefer)
df = df.fillna(0)

In [215]:
df

SPECIES_NAME,PROFILE_CODE,PROFILE_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,...,Selenium,Silicon,Sodium,Sodium ion,Sulfate,Sulfur,Tin,Vanadium,Zinc,Zirconium
0,3766AE6,Forest Fire - AE6,0.015400,0.110500,0.000500,0.000300,0.0,0.000100,0.369300,0.0,...,0.000000,0.062000,0.012200,0.0,0.133200,0.0792,0.001500,0.000100,0.002900,0.000000
1,5674,Marine Vessel - Main Engine - Heavy Fuel Oil,0.771625,0.000000,0.019000,0.000000,0.0,0.003958,0.259125,0.0,...,0.000000,0.000000,0.000000,0.0,38.032500,0.0000,0.013281,1.515625,0.018250,0.000000
2,5675AE6,Marine Vessel - Auxiliary Engine - Marine Gas ...,0.000000,0.000000,0.061466,0.000000,0.0,0.023641,1.912530,0.0,...,0.000000,0.340426,0.000000,0.0,12.078014,0.0000,0.035461,0.082742,0.200946,0.000000
3,8873AE6,Aircraft Exhaust,0.000000,1.791897,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,6.867400,0.0000,0.000000,0.000000,0.000000,0.000000
4,8898a,Outdoor Hydronic Heater- one stage updraft con...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.017300,0.010700,0.0,0.044100,0.0147,0.000000,0.000000,0.007760,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,95870,Catalytic Woodstove Startup Wood Composite,0.000000,0.000000,0.001704,0.000111,0.0,0.006554,0.056238,0.0,...,0.000223,0.026435,0.045670,0.0,0.417652,0.0000,0.010290,0.000154,0.045448,0.000927
229,95871,Catalytic Woodstove Burn Phase Composite Birch,0.000000,0.000000,0.010937,0.000005,0.0,0.006001,0.024024,0.0,...,0.001240,0.101690,0.021691,0.0,1.906428,0.0000,0.008238,0.000311,0.183862,0.000196
230,95872,Catalytic Woodstove Burn Phase Composite Spruce,0.000355,0.000000,0.000725,0.000006,0.0,0.002385,0.043209,0.0,...,0.000117,0.001333,0.003175,0.0,0.359246,0.0000,0.001160,0.000060,0.032112,0.000267
231,95873,Catalytic Woodstove Burn Phase and Wood Composite,0.000149,0.000000,0.006648,0.000006,0.0,0.004483,0.032082,0.0,...,0.000769,0.059540,0.013915,0.0,1.256611,0.0000,0.005265,0.000206,0.120127,0.000226


In [216]:
def clean_text(text):
    # Remove numbers and special characters
    text = re.sub(r'[^A-Za-z\s]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [217]:
# df['name'] = df['name'].apply(clean_text)

In [218]:
df

SPECIES_NAME,PROFILE_CODE,PROFILE_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,...,Selenium,Silicon,Sodium,Sodium ion,Sulfate,Sulfur,Tin,Vanadium,Zinc,Zirconium
0,3766AE6,Forest Fire - AE6,0.015400,0.110500,0.000500,0.000300,0.0,0.000100,0.369300,0.0,...,0.000000,0.062000,0.012200,0.0,0.133200,0.0792,0.001500,0.000100,0.002900,0.000000
1,5674,Marine Vessel - Main Engine - Heavy Fuel Oil,0.771625,0.000000,0.019000,0.000000,0.0,0.003958,0.259125,0.0,...,0.000000,0.000000,0.000000,0.0,38.032500,0.0000,0.013281,1.515625,0.018250,0.000000
2,5675AE6,Marine Vessel - Auxiliary Engine - Marine Gas ...,0.000000,0.000000,0.061466,0.000000,0.0,0.023641,1.912530,0.0,...,0.000000,0.340426,0.000000,0.0,12.078014,0.0000,0.035461,0.082742,0.200946,0.000000
3,8873AE6,Aircraft Exhaust,0.000000,1.791897,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,6.867400,0.0000,0.000000,0.000000,0.000000,0.000000
4,8898a,Outdoor Hydronic Heater- one stage updraft con...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.017300,0.010700,0.0,0.044100,0.0147,0.000000,0.000000,0.007760,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,95870,Catalytic Woodstove Startup Wood Composite,0.000000,0.000000,0.001704,0.000111,0.0,0.006554,0.056238,0.0,...,0.000223,0.026435,0.045670,0.0,0.417652,0.0000,0.010290,0.000154,0.045448,0.000927
229,95871,Catalytic Woodstove Burn Phase Composite Birch,0.000000,0.000000,0.010937,0.000005,0.0,0.006001,0.024024,0.0,...,0.001240,0.101690,0.021691,0.0,1.906428,0.0000,0.008238,0.000311,0.183862,0.000196
230,95872,Catalytic Woodstove Burn Phase Composite Spruce,0.000355,0.000000,0.000725,0.000006,0.0,0.002385,0.043209,0.0,...,0.000117,0.001333,0.003175,0.0,0.359246,0.0000,0.001160,0.000060,0.032112,0.000267
231,95873,Catalytic Woodstove Burn Phase and Wood Composite,0.000149,0.000000,0.006648,0.000006,0.0,0.004483,0.032082,0.0,...,0.000769,0.059540,0.013915,0.0,1.256611,0.0000,0.005265,0.000206,0.120127,0.000226


In [219]:
# numeric_cols = df.select_dtypes(include=[np.number]).columns
# text_cols = df.select_dtypes(include=[object]).columns

# print(text_cols)

# agg_funcs = {col: 'mean' for col in numeric_cols}
# agg_funcs.update({col: lambda x: np.random.choice(x) for col in text_cols if col != 'name'})

# pivot_df = df.groupby('name').agg(agg_funcs).reset_index()

In [220]:
# cols = list(pivot_df.columns)
# cols.insert(0, cols.pop(cols.index('code')))
# pivot_df = pivot_df[cols]
# pivot_df

In [221]:
pivot_df = df
pivot_df

SPECIES_NAME,PROFILE_CODE,PROFILE_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,...,Selenium,Silicon,Sodium,Sodium ion,Sulfate,Sulfur,Tin,Vanadium,Zinc,Zirconium
0,3766AE6,Forest Fire - AE6,0.015400,0.110500,0.000500,0.000300,0.0,0.000100,0.369300,0.0,...,0.000000,0.062000,0.012200,0.0,0.133200,0.0792,0.001500,0.000100,0.002900,0.000000
1,5674,Marine Vessel - Main Engine - Heavy Fuel Oil,0.771625,0.000000,0.019000,0.000000,0.0,0.003958,0.259125,0.0,...,0.000000,0.000000,0.000000,0.0,38.032500,0.0000,0.013281,1.515625,0.018250,0.000000
2,5675AE6,Marine Vessel - Auxiliary Engine - Marine Gas ...,0.000000,0.000000,0.061466,0.000000,0.0,0.023641,1.912530,0.0,...,0.000000,0.340426,0.000000,0.0,12.078014,0.0000,0.035461,0.082742,0.200946,0.000000
3,8873AE6,Aircraft Exhaust,0.000000,1.791897,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,6.867400,0.0000,0.000000,0.000000,0.000000,0.000000
4,8898a,Outdoor Hydronic Heater- one stage updraft con...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.017300,0.010700,0.0,0.044100,0.0147,0.000000,0.000000,0.007760,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,95870,Catalytic Woodstove Startup Wood Composite,0.000000,0.000000,0.001704,0.000111,0.0,0.006554,0.056238,0.0,...,0.000223,0.026435,0.045670,0.0,0.417652,0.0000,0.010290,0.000154,0.045448,0.000927
229,95871,Catalytic Woodstove Burn Phase Composite Birch,0.000000,0.000000,0.010937,0.000005,0.0,0.006001,0.024024,0.0,...,0.001240,0.101690,0.021691,0.0,1.906428,0.0000,0.008238,0.000311,0.183862,0.000196
230,95872,Catalytic Woodstove Burn Phase Composite Spruce,0.000355,0.000000,0.000725,0.000006,0.0,0.002385,0.043209,0.0,...,0.000117,0.001333,0.003175,0.0,0.359246,0.0000,0.001160,0.000060,0.032112,0.000267
231,95873,Catalytic Woodstove Burn Phase and Wood Composite,0.000149,0.000000,0.006648,0.000006,0.0,0.004483,0.032082,0.0,...,0.000769,0.059540,0.013915,0.0,1.256611,0.0000,0.005265,0.000206,0.120127,0.000226


In [222]:
pivot_df.iloc[:,2:47].head()

SPECIES_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,Chloride ion,Chlorine,...,Selenium,Silicon,Sodium,Sodium ion,Sulfate,Sulfur,Tin,Vanadium,Zinc,Zirconium
0,0.015400,0.110500,0.000500,0.0003,0.0,0.000100,0.369300,0.0,0.207,0.0,...,0.0,0.062000,0.0122,0.0,0.133200,0.0792,0.001500,0.000100,0.002900,0.0
1,0.771625,0.000000,0.019000,0.0000,0.0,0.003958,0.259125,0.0,0.000,0.0,...,0.0,0.000000,0.0000,0.0,38.032500,0.0000,0.013281,1.515625,0.018250,0.0
2,0.000000,0.000000,0.061466,0.0000,0.0,0.023641,1.912530,0.0,0.000,0.0,...,0.0,0.340426,0.0000,0.0,12.078014,0.0000,0.035461,0.082742,0.200946,0.0
3,0.000000,1.791897,0.000000,0.0000,0.0,0.000000,0.000000,0.0,0.000,0.0,...,0.0,0.000000,0.0000,0.0,6.867400,0.0000,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.0,0.000,0.0,...,0.0,0.017300,0.0107,0.0,0.044100,0.0147,0.000000,0.000000,0.007760,0.0


# Clustering based on numeric values

In [223]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from collections import Counter

In [224]:
def assign_cluster_name(cluster):
    names = pivot_df[pivot_df['speciated_cluster'] == cluster]['PROFILE_NAME']
    most_common_name = Counter(names).most_common(1)[0][0]
    return most_common_name

In [225]:
X = pivot_df.iloc[:, 2:45]
#scaler = StandardScaler()
# scaler = MinMaxScaler()
# X = scaler.fit_transform(X)

In [226]:
X

SPECIES_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,Chloride ion,Chlorine,...,Selenium,Silicon,Sodium,Sodium ion,Sulfate,Sulfur,Tin,Vanadium,Zinc,Zirconium
0,0.015400,0.110500,0.000500,0.000300,0.0,0.000100,0.369300,0.0,0.207000,0.0,...,0.000000,0.062000,0.012200,0.0,0.133200,0.0792,0.001500,0.000100,0.002900,0.000000
1,0.771625,0.000000,0.019000,0.000000,0.0,0.003958,0.259125,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,38.032500,0.0000,0.013281,1.515625,0.018250,0.000000
2,0.000000,0.000000,0.061466,0.000000,0.0,0.023641,1.912530,0.0,0.000000,0.0,...,0.000000,0.340426,0.000000,0.0,12.078014,0.0000,0.035461,0.082742,0.200946,0.000000
3,0.000000,1.791897,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,6.867400,0.0000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.017300,0.010700,0.0,0.044100,0.0147,0.000000,0.000000,0.007760,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,0.000000,0.000000,0.001704,0.000111,0.0,0.006554,0.056238,0.0,0.000000,0.0,...,0.000223,0.026435,0.045670,0.0,0.417652,0.0000,0.010290,0.000154,0.045448,0.000927
229,0.000000,0.000000,0.010937,0.000005,0.0,0.006001,0.024024,0.0,0.000000,0.0,...,0.001240,0.101690,0.021691,0.0,1.906428,0.0000,0.008238,0.000311,0.183862,0.000196
230,0.000355,0.000000,0.000725,0.000006,0.0,0.002385,0.043209,0.0,0.000000,0.0,...,0.000117,0.001333,0.003175,0.0,0.359246,0.0000,0.001160,0.000060,0.032112,0.000267
231,0.000149,0.000000,0.006648,0.000006,0.0,0.004483,0.032082,0.0,0.000000,0.0,...,0.000769,0.059540,0.013915,0.0,1.256611,0.0000,0.005265,0.000206,0.120127,0.000226


In [227]:
model = NMF(n_components=32, init='random', random_state=0, max_iter=1000)
W = model.fit_transform(X)
H = model.components_

# What features to use for clustering

In [228]:
# Assuming 'target' is the category column, and 'name' is the label
X = pivot_df.iloc[:, 2:45]
#X = W

In [229]:
n_clusters = 16
kmeans = KMeans(n_clusters=n_clusters, random_state=42,n_init='auto')
pivot_df['speciated_cluster'] = kmeans.fit_predict(X)

# Create a new column for cluster names
pivot_df['speciated_cluster_name'] = pivot_df['speciated_cluster'].apply(assign_cluster_name)

pivot_df.head()

SPECIES_NAME,PROFILE_CODE,PROFILE_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,...,Sodium,Sodium ion,Sulfate,Sulfur,Tin,Vanadium,Zinc,Zirconium,speciated_cluster,speciated_cluster_name
0,3766AE6,Forest Fire - AE6,0.015400,0.110500,0.000500,0.0003,0.0,0.000100,0.369300,0.0,...,0.0122,0.0,0.133200,0.0792,0.001500,0.000100,0.002900,0.0,1,Tire Dust
1,5674,Marine Vessel - Main Engine - Heavy Fuel Oil,0.771625,0.000000,0.019000,0.0000,0.0,0.003958,0.259125,0.0,...,0.0000,0.0,38.032500,0.0000,0.013281,1.515625,0.018250,0.0,13,Marine Vessel - Main Engine - Heavy Fuel Oil
2,5675AE6,Marine Vessel - Auxiliary Engine - Marine Gas ...,0.000000,0.000000,0.061466,0.0000,0.0,0.023641,1.912530,0.0,...,0.0000,0.0,12.078014,0.0000,0.035461,0.082742,0.200946,0.0,8,Marine Vessel - Auxiliary Engine - Marine Gas ...
3,8873AE6,Aircraft Exhaust,0.000000,1.791897,0.000000,0.0000,0.0,0.000000,0.000000,0.0,...,0.0000,0.0,6.867400,0.0000,0.000000,0.000000,0.000000,0.0,1,Tire Dust
4,8898a,Outdoor Hydronic Heater- one stage updraft con...,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.0,...,0.0107,0.0,0.044100,0.0147,0.000000,0.000000,0.007760,0.0,1,Tire Dust


In [230]:
# for cluster_num in range(n_clusters):
#     print(f"\nCluster {cluster_num}:")
#     cluster_texts = pivot_df[pivot_df['speciated_cluster'] == cluster_num]['PROFILE_NAME']
#     for text in cluster_texts:
#         print(f" - {text}")

# Hierarchical Clustering

In [231]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from collections import Counter
plt.rcParams['font.family'] = 'Sans'

In [232]:
# Assuming 'target' is the category column, and 'name' is the label
# X = pivot_df.iloc[:, 2:47]

# Perform Agglomerative Clustering with more branches (e.g., 5 clusters)
max_clusters = 10
#clustering = AgglomerativeClustering(n_clusters=max_clusters, linkage='ward')
clustering = AgglomerativeClustering(n_clusters=max_clusters, linkage='ward', metric='euclidean')
pivot_df['cluster'] = clustering.fit_predict(X)

# Assign majority name within each cluster
def get_majority_label(cluster, names):
    return Counter(names).most_common(1)[0][0]

cluster_majority_labels = {
    cluster: get_majority_label(cluster, pivot_df[pivot_df['cluster'] == cluster]['PROFILE_NAME'])
    for cluster in range(max_clusters)
}

# Replace cluster numbers with majority names
pivot_df['cluster_label'] = pivot_df['cluster'].map(cluster_majority_labels)

# Plot the clusters (using seaborn scatter plot for visualization)
plt.figure(figsize=(10, 7))

# Initialize NMF and fit to the data
# model = NMF(n_components=2, init='random', random_state=0)
# W = model.fit_transform(X)
# H = model.components_


sns.scatterplot(data=pivot_df, x=W[:, 0], y=W[:, 1], hue='cluster_label', palette='tab10', s=100)
plt.title(f'Clustering with {max_clusters} Branches')
plt.show()


## Dendogram Hierarchy

In [233]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from collections import Counter
from scipy.cluster.hierarchy import dendrogram, linkage

# Assuming 'target' is the category column, and 'name' is the label
# X = pivot_df.iloc[:, 2:47]

# Perform Agglomerative Clustering with more branches (e.g., 5 clusters)
max_clusters = 24
clustering = AgglomerativeClustering(n_clusters=max_clusters, linkage='ward')
pivot_df['cluster'] = clustering.fit_predict(X)

# Assign majority name within each cluster
def get_majority_label(cluster, names):
    return Counter(names).most_common(1)[0][0]

cluster_majority_labels = {}
for cluster in range(max_clusters):
    cluster_names = pivot_df[pivot_df['cluster'] == cluster]['PROFILE_NAME']
    majority_label = get_majority_label(cluster, cluster_names)
    cluster_majority_labels[cluster] = majority_label
    # Print cluster representative name and count of items in cluster
    print(f"Cluster {cluster}: Representative Name = {majority_label}, Number of Items = {len(cluster_names)}")

# Replace cluster numbers with majority names
pivot_df['cluster_label'] = pivot_df['cluster'].map(cluster_majority_labels)

# Generate the linkage matrix
Z = linkage(X, method='ward')

# Plot the circular dendrogram
plt.figure(figsize=(10, 7))
dendrogram(Z, truncate_mode='lastp', p=max_clusters, labels=pivot_df['cluster_label'].values, leaf_font_size=10, orientation='right')
plt.title(f'Circular Dendrogram of Hierarchical Clustering with {max_clusters} Branches')
plt.show()


Cluster 0: Representative Name = Marine Vessel - Main Engine - Heavy Fuel Oil, Number of Items = 7
Cluster 1: Representative Name = Marine Vessel - Auxiliary Engine - Marine Gas Oil  AE6, Number of Items = 22
Cluster 2: Representative Name = Lignite Combustion - Composite, Number of Items = 11
Cluster 3: Representative Name = Brake Wear, Number of Items = 13
Cluster 4: Representative Name = Kraft Recovery Furnace - Composite, Number of Items = 4
Cluster 5: Representative Name = Residential Wood Combustion - non-EPA Certified Woodstove, Almond, Number of Items = 9
Cluster 6: Representative Name = Agricultural Burning - Wheat Straw - China, Number of Items = 5
Cluster 7: Representative Name = Unpaved Road Dust - Composite, Number of Items = 19
Cluster 8: Representative Name = Residential Wood Combustion - Fireplace, Oak, Number of Items = 4
Cluster 9: Representative Name = Brake Wear, Number of Items = 6
Cluster 10: Representative Name = Distillate Oil Combustion - Composite, Number of 

### Dendogram of all points

In [234]:
# plt.figure(figsize=(10, 100))
# dendrogram(Z, labels=pivot_df['PROFILE_NAME'].values, leaf_font_size=10, orientation='right')
# plt.title(f'Circular Dendrogram of Hierarchical Clustering with {max_clusters} Branches')
# plt.show()


## Print Top k representatives within each clusters

In [235]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from collections import Counter
from scipy.cluster.hierarchy import dendrogram, linkage

# Assuming 'target' is the category column, and 'name' is the label
# X = pivot_df.iloc[:, 2:47]

# Perform Agglomerative Clustering with more branches (e.g., 20 clusters)
max_clusters = 10
clustering = AgglomerativeClustering(n_clusters=max_clusters, linkage='ward')
pivot_df['cluster'] = clustering.fit_predict(X)

# Modify the get_majority_label function to return top k labels with their counts
def get_top_k_labels(cluster, names, k=3):
    counter = Counter(names).most_common(k)
    return [(label, count) for label, count in counter]

cluster_majority_labels = {}
k = 1  # Choose the number of majority names to show

for cluster in range(max_clusters):
    cluster_names = pivot_df[pivot_df['cluster'] == cluster]['PROFILE_NAME']
    top_k_labels = get_top_k_labels(cluster, cluster_names, k)
    
    # Combine top k labels into a string format 'Label1 (Count1), Label2 (Count2), ...'
    majority_label_str = ', '.join([f"{label} ({count})" for label, count in top_k_labels])
    
    cluster_majority_labels[cluster] = majority_label_str
    
    # Print cluster representative names and counts
    print(f"Cluster {cluster}: Representative Names = {majority_label_str}, Number of Items = {len(cluster_names)}")
    print("")

# Replace cluster numbers with majority names and their counts
pivot_df['cluster_label'] = pivot_df['cluster'].map(cluster_majority_labels)

# Generate the linkage matrix
Z = linkage(X, method='ward')

# Plot the circular dendrogram with updated labels
plt.figure(figsize=(10, 7))
dendrogram(Z, truncate_mode='lastp', p= max_clusters, labels=pivot_df['cluster_label'].values, leaf_font_size=10, orientation='right')
plt.title(f'Circular Dendrogram of Hierarchical Clustering with {max_clusters} Branches')
plt.show()


Cluster 0: Representative Names = Marine Vessel - Auxiliary Engine - Marine Gas Oil  AE6 (1), Number of Items = 33

Cluster 1: Representative Names = Residential Wood Combustion - Fireplace, Juniper (3), Number of Items = 18

Cluster 2: Representative Names = Paved Road Dust (2), Number of Items = 29

Cluster 3: Representative Names = Brake Wear (12), Number of Items = 15

Cluster 4: Representative Names = Marine Vessel - Main Engine - Heavy Fuel Oil (1), Number of Items = 11

Cluster 5: Representative Names = Ammonium Nitrate Production - Composite (1), Number of Items = 1

Cluster 6: Representative Names = Lignite Combustion - Composite (1), Number of Items = 11

Cluster 7: Representative Names = Brick Making Kiln (1), Number of Items = 6

Cluster 8: Representative Names = Brake Wear (8), Number of Items = 105

Cluster 9: Representative Names = Kraft Recovery Furnace - Composite (1), Number of Items = 4



## k-NN Graph Construction

In [236]:
from sklearn.neighbors import NearestNeighbors
import networkx as nx
from scipy.sparse.linalg import svds

In [237]:
# Apply SVD to X
U, S, VT = svds(X.values, k=16)
X_svd = U @ np.diag(S) @ VT

In [238]:
X = X_svd

In [239]:
# Compute KNN

K = 3
knn = NearestNeighbors(n_neighbors=K)  # You can change the number of neighbors
knn.fit(X)
distances, indices = knn.kneighbors(X)

# Create a graph
G = nx.Graph()

# Add nodes with PROFILE_NAME as labels
for i, profile_name in enumerate(df['PROFILE_NAME']):
    G.add_node(i, label=profile_name)

# Add edges based on KNN
for i, neighbors in enumerate(indices):
    for neighbor in neighbors:
        if i != neighbor:  # Avoid self-loops
            G.add_edge(i, neighbor)

In [240]:
import matplotlib
matplotlib.use('Agg')  # Use a non-interactive backend

In [241]:
# pos = nx.spring_layout(G)  # Layout for visualization
# labels = nx.get_node_attributes(G, 'label')

# plt.figure(figsize=(50, 50))
# nx.draw(G, pos, with_labels=True, labels=labels, node_size=100, node_color='skyblue', font_size=10, font_color='black', edge_color='gray')
# plt.title('K-Nearest Neighbor Graph')
# plt.show()

# Community Dectection

In [242]:
import cdlib
from cdlib import algorithms
import networkx as nx
# G = nx.karate_club_graph()

# non-overlap louvain
coms = algorithms.louvain(G, weight='weight', resolution=1., randomize=False)

In [243]:
print(coms.method_name)
print(coms.communities)
print(coms.node_coverage)
print(coms.overlap )
print(coms.method_parameters)
#print(coms.to_json)
print(coms.average_internal_degree())
print(coms.average_internal_degree(summary=False))

Louvain
[[12, 13, 19, 20, 23, 41, 46, 49, 52, 69, 71, 73, 81, 86, 112, 113, 114, 115, 160, 164, 167, 168, 169, 170, 171, 172, 174, 175, 176, 177, 178, 200, 201], [4, 18, 25, 28, 60, 64, 82, 119, 120, 121, 122, 123, 124, 125, 126, 147, 148, 154, 204, 208, 209, 216], [100, 101, 102, 103, 104, 106, 107, 156, 157, 158, 159, 203, 205, 207, 210, 211, 212, 214, 219, 232], [0, 17, 117, 118, 151, 152, 153, 206, 213, 215, 217, 218, 221, 223, 224, 225, 228, 230], [182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199], [16, 22, 37, 38, 39, 48, 53, 72, 76, 84, 88, 89, 90, 93, 109, 110, 155], [5, 26, 42, 66, 70, 91, 127, 132, 134, 136, 137, 173, 179, 180, 181], [6, 14, 15, 40, 44, 51, 58, 63, 87, 220, 222, 226, 227, 229, 231], [1, 11, 29, 31, 35, 45, 50, 55, 56, 78, 85, 92, 111, 163], [2, 3, 8, 9, 24, 27, 32, 33, 57, 59, 68, 80, 108, 202], [7, 10, 34, 36, 61, 62, 67, 74, 77, 79, 94, 161, 162], [65, 129, 133, 135, 139, 143, 144, 145, 146, 150], [21, 30, 43, 47, 54,

In [244]:
com_map={}

for i in range(len(coms.communities)):
    community = coms.communities[i]
    for j in community:
        com_map[j] = i
        
com_map

{12: 0,
 13: 0,
 19: 0,
 20: 0,
 23: 0,
 41: 0,
 46: 0,
 49: 0,
 52: 0,
 69: 0,
 71: 0,
 73: 0,
 81: 0,
 86: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 0,
 160: 0,
 164: 0,
 167: 0,
 168: 0,
 169: 0,
 170: 0,
 171: 0,
 172: 0,
 174: 0,
 175: 0,
 176: 0,
 177: 0,
 178: 0,
 200: 0,
 201: 0,
 4: 1,
 18: 1,
 25: 1,
 28: 1,
 60: 1,
 64: 1,
 82: 1,
 119: 1,
 120: 1,
 121: 1,
 122: 1,
 123: 1,
 124: 1,
 125: 1,
 126: 1,
 147: 1,
 148: 1,
 154: 1,
 204: 1,
 208: 1,
 209: 1,
 216: 1,
 100: 2,
 101: 2,
 102: 2,
 103: 2,
 104: 2,
 106: 2,
 107: 2,
 156: 2,
 157: 2,
 158: 2,
 159: 2,
 203: 2,
 205: 2,
 207: 2,
 210: 2,
 211: 2,
 212: 2,
 214: 2,
 219: 2,
 232: 2,
 0: 3,
 17: 3,
 117: 3,
 118: 3,
 151: 3,
 152: 3,
 153: 3,
 206: 3,
 213: 3,
 215: 3,
 217: 3,
 218: 3,
 221: 3,
 223: 3,
 224: 3,
 225: 3,
 228: 3,
 230: 3,
 182: 4,
 183: 4,
 184: 4,
 185: 4,
 186: 4,
 187: 4,
 188: 4,
 189: 4,
 190: 4,
 191: 4,
 192: 4,
 193: 4,
 194: 4,
 195: 4,
 196: 4,
 197: 4,
 198: 4,
 199: 4,
 16: 5,
 22: 5,
 37: 5,
 3

In [245]:
coms_l = [algorithms.louvain(G), algorithms.label_propagation(G), algorithms.walktrap(G)]
overlap_coms = algorithms.endntm(G, coms_l)

In [246]:
print(overlap_coms.communities)

[[69, 71, 200, 73, 201, 12, 13, 81, 19, 20, 86, 23, 164, 167, 168, 169, 170, 171, 172, 173, 46, 174, 175, 176, 177, 178, 52, 49], [64, 0, 4, 204, 208, 209, 18, 147, 148, 82, 216, 25, 154, 28, 95, 124, 119, 120, 121, 122, 123, 60, 125, 126], [0, 206, 17, 82, 213, 215, 151, 217, 218, 152, 153, 221, 223, 224, 225, 228, 230, 117, 118], [192, 193, 194, 195, 196, 197, 198, 199, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191], [72, 76, 16, 84, 22, 88, 89, 90, 155, 93, 37, 38, 39, 109, 110, 48, 53], [1, 35, 163, 11, 45, 78, 111, 50, 53, 85, 55, 56, 87, 92, 29, 31], [66, 132, 5, 70, 134, 136, 137, 42, 173, 46, 179, 180, 181, 26, 91, 127], [226, 227, 26, 229, 6, 231, 40, 44, 14, 15, 51, 87, 58, 220, 222, 63], [32, 33, 2, 3, 68, 8, 9, 59, 202, 108, 80, 24, 57, 27], [100, 101, 102, 103, 104, 232, 106, 107, 205, 210, 156, 157, 158, 159], [161, 34, 67, 36, 162, 7, 10, 74, 77, 79, 94, 61, 62], [65, 129, 133, 135, 139, 143, 144, 145, 146, 150], [116, 21, 54, 75, 43, 30, 47], [160, 112, 113, 114, 49,

In [247]:
overlap_com_map={}

for i in range(len(overlap_coms.communities)):
    community = overlap_coms.communities[i]
    for j in community:
        if j in overlap_com_map:
            overlap_com_map[j].append(i)
        else:
            overlap_com_map[j] = [i]
            
overlap_com_map

{69: [0],
 71: [0],
 200: [0],
 73: [0],
 201: [0],
 12: [0],
 13: [0],
 81: [0],
 19: [0],
 20: [0],
 86: [0],
 23: [0],
 164: [0],
 167: [0],
 168: [0],
 169: [0],
 170: [0],
 171: [0],
 172: [0],
 173: [0, 6],
 46: [0, 6],
 174: [0],
 175: [0],
 176: [0],
 177: [0],
 178: [0],
 52: [0],
 49: [0, 13],
 64: [1],
 0: [1, 2],
 4: [1],
 204: [1],
 208: [1],
 209: [1],
 18: [1],
 147: [1],
 148: [1],
 82: [1, 2],
 216: [1],
 25: [1],
 154: [1],
 28: [1],
 95: [1, 15],
 124: [1],
 119: [1],
 120: [1],
 121: [1],
 122: [1],
 123: [1],
 60: [1],
 125: [1],
 126: [1],
 206: [2, 14],
 17: [2],
 213: [2],
 215: [2],
 151: [2],
 217: [2],
 218: [2],
 152: [2],
 153: [2],
 221: [2],
 223: [2],
 224: [2],
 225: [2],
 228: [2],
 230: [2],
 117: [2],
 118: [2],
 192: [3],
 193: [3],
 194: [3],
 195: [3],
 196: [3],
 197: [3],
 198: [3],
 199: [3],
 182: [3],
 183: [3],
 184: [3],
 185: [3],
 186: [3],
 187: [3],
 188: [3],
 189: [3],
 190: [3],
 191: [3],
 72: [4],
 76: [4],
 16: [4],
 84: [4],
 22:

In [248]:
for i in range(len(pivot_df)):
    if i not in overlap_com_map:
        print(i)

In [249]:
# nx.write_graphml(G, 'knn_graph.graphml')


labels=[]
# y = [i+'-'+j for i,j in zip(pivot_df['PROFILE_NAME'],pivot_df['PROFILE_CODE'])]
y = [i for i in pivot_df['PROFILE_CODE']]
labels = dict(zip(range(len(y)), y))

# print(labels)

overlap_com_map_str = {key: ",".join([str(i) for i in value]) for key, value in overlap_com_map.items()}

nx.set_node_attributes(G, labels, 'labels')
nx.set_node_attributes(G, com_map, 'non_overlap_community')
nx.set_node_attributes(G, overlap_com_map_str, 'overlap_community')

filename = DATASET_NAME+"KNN"+str(K)+'.gexf'
print("Writing gephi...."+filename)
nx.write_gexf(G, filename)
print("Done....")

Writing gephi....SpeciateV5.3_PM_AE6.csvKNN3.gexf
Done....


## Data preprocessing

In [250]:
category_for_classification = 'speciated_cluster_name' 
#'assigned_profile', #'speciated_cluster_name'

In [251]:
# # Step 1: Read the CSV into a pandas dataframe

# filename = 'PM25-Speciated/IMPROVEDataset/usepa_final_with_assigned_profile.csv'
# df = pd.read_csv(filename)

In [252]:
# df

In [253]:
# Create a dictionary to map categories to numerical values
category_mapping = {category: index for index, category in enumerate(pivot_df[category_for_classification].unique())}

In [254]:
# Replace string categories with numerical values
pivot_df['final_profile'] = pivot_df[category_for_classification].map(category_mapping)

# The updated DataFrame and the mapping dictionary
print(category_mapping)
pivot_df

{'Tire Dust': 0, 'Marine Vessel - Main Engine - Heavy Fuel Oil': 1, 'Marine Vessel - Auxiliary Engine - Marine Gas Oil \x96 AE6': 2, 'Diesel Exhaust - Heavy-heavy duty truck - 2007 model year with NCOM': 3, 'Paved Road Dust': 4, 'Kraft Recovery Furnace - Composite': 5, 'Lignite Combustion - Composite': 6, 'Brake Wear': 7, 'Catalytic Cracking - Composite': 8, 'Ammonium Nitrate Production - Composite': 9, 'Sea Salt - Composite': 10, 'Lead Production - Composite': 11, 'Brick Making Kiln': 12, 'Agricultural Burning - Wheat Straw - China': 13, 'Residential Wood Combustion - Fireplace, Juniper': 14}


SPECIES_NAME,PROFILE_CODE,PROFILE_NAME,Aluminum,Ammonium,Antimony,Arsenic,Bromine,Cadmium,Calcium,Calcium ion,...,Sulfur,Tin,Vanadium,Zinc,Zirconium,speciated_cluster,speciated_cluster_name,cluster,cluster_label,final_profile
0,3766AE6,Forest Fire - AE6,0.015400,0.110500,0.000500,0.000300,0.0,0.000100,0.369300,0.0,...,0.0792,0.001500,0.000100,0.002900,0.000000,1,Tire Dust,8,Brake Wear (8),0
1,5674,Marine Vessel - Main Engine - Heavy Fuel Oil,0.771625,0.000000,0.019000,0.000000,0.0,0.003958,0.259125,0.0,...,0.0000,0.013281,1.515625,0.018250,0.000000,13,Marine Vessel - Main Engine - Heavy Fuel Oil,4,Marine Vessel - Main Engine - Heavy Fuel Oil (1),1
2,5675AE6,Marine Vessel - Auxiliary Engine - Marine Gas ...,0.000000,0.000000,0.061466,0.000000,0.0,0.023641,1.912530,0.0,...,0.0000,0.035461,0.082742,0.200946,0.000000,8,Marine Vessel - Auxiliary Engine - Marine Gas ...,0,Marine Vessel - Auxiliary Engine - Marine Gas ...,2
3,8873AE6,Aircraft Exhaust,0.000000,1.791897,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0000,0.000000,0.000000,0.000000,0.000000,1,Tire Dust,0,Marine Vessel - Auxiliary Engine - Marine Gas ...,0
4,8898a,Outdoor Hydronic Heater- one stage updraft con...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0147,0.000000,0.000000,0.007760,0.000000,1,Tire Dust,8,Brake Wear (8),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,95870,Catalytic Woodstove Startup Wood Composite,0.000000,0.000000,0.001704,0.000111,0.0,0.006554,0.056238,0.0,...,0.0000,0.010290,0.000154,0.045448,0.000927,1,Tire Dust,8,Brake Wear (8),0
229,95871,Catalytic Woodstove Burn Phase Composite Birch,0.000000,0.000000,0.010937,0.000005,0.0,0.006001,0.024024,0.0,...,0.0000,0.008238,0.000311,0.183862,0.000196,1,Tire Dust,8,Brake Wear (8),0
230,95872,Catalytic Woodstove Burn Phase Composite Spruce,0.000355,0.000000,0.000725,0.000006,0.0,0.002385,0.043209,0.0,...,0.0000,0.001160,0.000060,0.032112,0.000267,1,Tire Dust,8,Brake Wear (8),0
231,95873,Catalytic Woodstove Burn Phase and Wood Composite,0.000149,0.000000,0.006648,0.000006,0.0,0.004483,0.032082,0.0,...,0.0000,0.005265,0.000206,0.120127,0.000226,1,Tire Dust,8,Brake Wear (8),0


In [255]:
len(category_mapping)

15

In [256]:
mapping_category = {value:key for key, value in category_mapping.items()}

In [257]:
# Step 2: Apply Non-negative matrix factorization on the data except last column
# Handle missing values by filling them with the mean of the column
# df.fillna(df.mean(), inplace=True)

In [258]:
# Separate features and target
X = pivot_df.iloc[:, 2:45].values
y = pivot_df['final_profile'].values

In [259]:
print(X.shape)
print(y.shape)

(233, 43)
(233,)


## NMF

In [260]:
# # Initialize NMF and fit to the data
# model = NMF(n_components=2, init='random', random_state=0)
# W = model.fit_transform(X)
# H = model.components_

# # Visualize the data
# plt.scatter(W[:, 0], W[:, 1], c=y)
# plt.xlabel('Component 1')
# plt.ylabel('Component 2')
# plt.title('NMF Components')
# plt.colorbar()
# plt.show()

## MLP

In [261]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import random
import numpy as np

In [262]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Example usage
set_seed(42)

In [263]:
# Step 3: Use PyTorch MLP to classify the last column
# Preprocess the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [264]:
X_scaled = torch.FloatTensor(X_scaled)
y = torch.LongTensor(y)

In [265]:
# Assuming X and y are already defined as PyTorch tensors
# X = features tensor
# y = labels tensor

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [266]:
print(X_train.shape)
print(X_test.shape)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

torch.Size([186, 43])
torch.Size([47, 43])


In [267]:

# Define a simple MLP model for multi-class classification
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(X_train.shape[1], 256),
            nn.ReLU(),
            nn.Linear(256, len(category_mapping))  # Adjusted for 29 classes
        )
        
    def forward(self, x):
        return self.layers(x)

# Initialize the model
model = MLP()

In [268]:
model

MLP(
  (layers): Sequential(
    (0): Linear(in_features=43, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=15, bias=True)
  )
)

In [269]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = torch.optim.Adam(model.parameters())

In [270]:
def train():
    # Training loop
    for epoch in range(1000):  # number of epochs can be adjusted
        for batch_idx, (data, target) in enumerate(train_loader):
            # Forward pass
            output = model(data)
            loss = criterion(output, target)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % 50 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

# train()

In [271]:
def test():
    model.eval()
    test_loss = 0
    all_targets = []
    all_predictions = []

    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            all_targets.extend(target.tolist())
            all_predictions.extend(output.argmax(dim=1).tolist())

    test_loss /= len(test_loader.dataset)
    print(f"Test Loss: {test_loss}")

    # Calculate F1 score
    f1 = f1_score(all_targets, all_predictions, average='weighted')
    print(f"F1 Score: {f1}")
    
    return all_predictions, all_targets
    
# pred, target = test()

In [272]:
# for i, j in zip(pred, target):
#     print("Pred:",mapping_category[i],"\n","True:",mapping_category[j])
#     print()

In [273]:
a = [1, 2, 3]
",".join([str(i) for i in a])

'1,2,3'